In [9]:
from melgan.model.generator import Generator
from melgan.utils.hparams import HParam
import torch
import soundfile as sf
import os
import librosa
import numpy as np

In [43]:
def audio_to_spectrogram(src_dir):
    spectrogram = []
    for root, _, files in os.walk(src_dir, topdown=False):
        for file_name in files:
            if file_name.endswith('.flac') or file_name.endswith('.mp3'):
                try:
                    print(file_name)
                    input_file = os.path.join(root, file_name)
                    audio, sr = librosa.load(input_file, sr=22050)
                    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=1024, hop_length=256, n_mels=80)
                    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
                    if log_mel_spectrogram.shape[-1] >= 64:    # training sample consists of 64 randomly cropped frames
                        return log_mel_spectrogram
                except AttributeError as e:
                    print(f"Error processing file: {file_name} - {e}")
                    continue
    return spectrogram

In [48]:
checkpoint = torch.load("./nvidia_tacotron2_LJ11_epoch6400.pt") 
hp = HParam("./melgan/config/default.yaml")

vocoder = Generator(hp.audio.n_mel_channels).cuda()
vocoder.load_state_dict(checkpoint["model_g"])
vocoder.eval()
# mel = torch.randn(1, 80, 234) # use your own mel-spectrogram here
# print(mel.shape)
spec = librosa.db_to_power(audio_to_spectrogram('../iu'))
mel = torch.from_numpy(spec[np.newaxis,:])

if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    mel = mel.cuda()

with torch.no_grad():
    audio = vocoder.inference(mel)
    
print(audio)
sf.write("audio2.wav", librosa.feature.inverse.mel_to_audio(spec, sr=22050, n_fft=1024, hop_length=256), 22050)
sf.write("audio.wav", audio.cpu().numpy(), 22050)

How Far I_ll Go.mp3
tensor([ -1771,  -7859, -14237,  ...,     30,     10,    -27], device='cuda:0',
       dtype=torch.int16)


## Conclusion: librosa sounds best